In [2]:
from markdown_it.rules_block import table
!pip install python-dotenv
!pip install lxml
!pip install lxml[html_clean]

In [ ]:
# pip install mysql-connector-python <- 터미널에 설치

In [2]:
!pip install requests

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")
## 실제로 찍으면안됨

if not NAVER_CLIENT_ID or not NAVER_CLIENT_SECRET:
    raise ValueError("NAVER_CLIENT_ID and NAVER_CLIENT_SECRET 이 설정되지 않았습니다")

## 테이블/모델 설계
테이블과 매칭되는 **엔티티클래스**를 설계한다.

```sql
use team5_db;
create table team5_db.transit_news (
    id int primary key auto_increment,
    title varchar(512),
    originallink varchar(512) UNIQUE,
    link varchar(512),
    description text,
    pub_date varchar(512),
    created_at datetime default (current_timestamp)
);
```

In [5]:
from datetime import datetime

"""
naver_news = NaverNews()
print(naver_news.title) # @property getter 메소드
naver_news.title = "새 제목" # @title.setter 메소드
"""

class NaverNews:
    def __init__(self, id: int, title: str, originallink: str, link: str, description: str, pubDate: str, created_at: datetime=None):
        self.__id = id
        self.__title = title
        self.__originallink = originallink
        self.__link = link
        self.__description = description
        self.__pubDate = pubDate
        self.__created_at = created_at

    @property
    def id(self):
        return self.__id

    @property
    def title(self):
        return self.__title

    # __title속성에 대한 setter 메소드
    # @title.setter
    # def title(self, value):
    #     self.__title = value

    @property
    def originallink(self):
        return self.__originallink

    @property
    def link(self):
        return self.__link

    @property
    def description(self):
        return self.__description

    @property
    def pubDate(self):
        return self.__pubDate

    @property
    def created_at(self):
        return self.__created_at

    def __repr__(self):
        return f'NaverNews({self.__id}, {self.__title}, {self.__originallink}, {self.__link}, {self.__description}, {self.__pubDate}, {self.__created_at})'

In [22]:
import requests
from pprint import pprint

url = 'https://openapi.naver.com/v1/search/news.json'

headers = {
    "X-Naver-Client-Id": NAVER_CLIENT_ID,
    "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
}
user_input = input('교통사고입력')
# 사용자입력값 (query string)
params = {
    'query': user_input,
    'display': 3, # 10 ~ 100
    'start': 1,
    'sort': 'sim', # sim | date
}

# 요청
response = requests.get(url, headers=headers, params=params)

naver_news_list: list[NaverNews] = []

# 결과출력
if response.status_code == 200:
    # print(response.text) # json문자열 확인
    data = response.json() # json형식의 데이터를 dict 변환
    items = data['items']
    for item in items:
        naver_news_list.append(NaverNews(None, **item))
    print(naver_news_list)
else:
    print("Error Code:" + response.status_code)

[NaverNews(None, <b>서울시</b>, 청계천서 수질오염<b>사고</b> 대비 방제훈련, https://www.newsis.com/view/NISX20250424_0003153003, https://n.news.naver.com/mnews/article/003/0013205255?sid=102, <b>서울시</b>는 지난 24일 성동구 신답역 청계천 하류에서 '수질오염<b>사고</b> 대응 방제훈련'을 실시했다고 25일... 현장 훈련은 차량 <b>교통사고</b>로 인해 유류가 하천으로 유입된 상황을 가정해 <b>사고</b> 접수 및 상황전파·보고... , Fri, 25 Apr 2025 06:00:00 +0900, None), NaverNews(None, 수질오염 <b>사고</b> 때 대응능력 강화…<b>서울시</b>, 청계천서 방제훈련, https://www.yna.co.kr/view/AKR20250424153400004?input=1195m, https://n.news.naver.com/mnews/article/001/0015351002?sid=102, <b>서울시</b>는 전날 성동구 신답역 청계천 하류에서 '수질오염<b>사고</b> 대응 방제훈련'을 했다고 25일 밝혔다. 이번... 현장훈련은 차량 <b>교통사고</b>로 인해 유류가 하천으로 유입된 상황을 가정해 ▲ <b>사고</b> 접수 및 상황전파·보고... , Fri, 25 Apr 2025 06:00:00 +0900, None), NaverNews(None, <b>서울시</b>, 청계천서 수질오염<b>사고</b> 대비 방제훈련 실시, http://news.mt.co.kr/mtview.php?no=2025042418172719006, https://n.news.naver.com/mnews/article/008/0005185780?sid=102, 이번 훈련에는 <b>서울시</b>와 자치구 담당자를 비롯해 미래한강본부, 서울아리수본부, <b>서울시</b>설공단, 한국환경공단, <b>서울시<

In [8]:
import mysql.connector

db_config = {
    'host': os.getenv('DB_HOST'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_NAME')
}

In [24]:

try:
    with mysql.connector.connect(**config) as conn:
        with conn.cursor() as cursor:
            # naver_news객체를 한번에 한건씩 insert처리
            for naver_news in naver_news_list:
                cursor.execute('''
                    insert into
                        naver_news (title, originallink, link, description, pub_date)
                    values (%s, %s, %s, %s, %s)
                ''', (naver_news.title, naver_news.originallink, naver_news.link, naver_news.description, naver_news.pubDate)) # sql, params
            # 트랜잭션 처리를 위해 모든 dml 작업후 마지막에 처리
            conn.commit()
except mysql.connector.Error as e:
    print('DB 오류:', e)

In [9]:
try:
    with mysql.connector.connect(**config) as conn:
        with conn.cursor() as cursor:
            # 테이블 안에 있는 모든 데이터 삭제
            cursor.execute('DELETE FROM naver_news')

            # 삭제한 내용을 데이터베이스에 확정
            conn.commit()
except mysql.connector.Error as e:
    print('DB 오류:', e)

In [26]:
# import openai -- > 실패
# from newspaper import Article
#
# # OpenAI API 키 설정
# openai.api_key =-OPENAI-API-KEY'
#
# import requests
# from bs4 import BeautifulSoup
#
# def get_naver_article_text(url):
#     headers = {'User-Agent': 'Mozilla/5.0'}
#     res = requests.get(url, headers=headers)
#     soup = BeautifulSoup(res.text, 'html.parser')
#     # 본문이 들어있는 div#dic_area
#     article_div = soup.find('div', id='dic_area')
#     if not article_div:
#         raise ValueError("본문 영역을 찾을 수 없습니다.")
#     # p 태그 텍스트만 모아서
#     paragraphs = [p.get_text(strip=True) for p in article_div.find_all('p')]
#     return "\n".join(paragraphs)

```sql
drop table transit_news
```
